# colab

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/bias gpt")
import os
os.environ['WANDB_DISABLED'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Mounted at /content/gdrive


In [2]:
!pip install datasets
!pip install transformers==4.43

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 61.7 MB/s eta 0:00:00
   ━

# Head

In [ ]:
import torch, importlib
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import GPT2TokenizerFast, DistilBertTokenizerFast, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import matplotlib.pyplot as plt
from datasets import load_dataset
from torch.utils.data import random_split


model_name = "distilgpt2"
gpt_tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
gpt_tokenizer.pad_token = gpt_tokenizer.eos_token

# 原神

In [24]:
prompt = "\n"
sentence = prompt + open('Genshin Impact.txt').read().replace('\n', ' ')
gpt_inputs = gpt_tokenizer(sentence, return_tensors="pt", max_length = 128, truncation = True)

In [26]:
import TextVAE
importlib.reload(TextVAE)
from TextVAE import TextVAE

tvae = TextVAE()
if torch.cuda.is_available():
    tvae = tvae.cuda()
    TextVAE.gpt.cuda()
tvae.train()
epochs = 100
optim = AdamW(tvae.parameters(), 5e-4)
scheduler = CosineAnnealingLR(optim, T_max = epochs, eta_min = 0)
for epoch in range(epochs):
    loss = tvae(gpt_inputs['input_ids'], gpt_inputs['attention_mask'])[0]
    loss.backward()
    optim.step()
    optim.zero_grad()
    scheduler.step()
    print(f"epoch: {epoch + 1}; loss: {loss.item()}")

4.893867492675781 800.150634765625
epoch: 1; loss: 4.893867492675781
5.428919792175293 3503.69921875
epoch: 2; loss: 5.428919792175293
4.753907203674316 7782.6025390625
epoch: 3; loss: 4.753907203674316
3.5386476516723633 12785.26171875
epoch: 4; loss: 3.5386476516723633
2.8089845180511475 18029.33203125
epoch: 5; loss: 2.8089845180511475
2.3788206577301025 23271.25390625
epoch: 6; loss: 2.3788206577301025
2.0285370349884033 28376.013671875
epoch: 7; loss: 2.0285370349884033
1.7641745805740356 33273.70703125
epoch: 8; loss: 1.7641745805740356
1.5894039869308472 37925.40625
epoch: 9; loss: 1.5894039869308472
1.4102774858474731 42310.0703125
epoch: 10; loss: 1.4102774858474731
1.2389942407608032 46419.3203125
epoch: 11; loss: 1.2389942407608032
1.127394437789917 50253.7890625
epoch: 12; loss: 1.127394437789917
1.0218123197555542 53819.1328125
epoch: 13; loss: 1.0218123197555542
0.9178276658058167 57123.95703125
epoch: 14; loss: 0.9178276658058167
0.8388864398002625 60178.5078125
epoch: 1

In [ ]:
logits = tvae(gpt_inputs['input_ids'], gpt_inputs['attention_mask'], do_sample = False)[2]
print(gpt_tokenizer.decode(logits.argmax(-1)[0]))
print((logits.argmax(-1)[0] != gpt_inputs['input_ids'][0, 1:]).sum().item())

In [ ]:
tvae.eval()
bias = tvae(gpt_inputs['input_ids'], gpt_inputs['attention_mask'], do_sample = True)[1]
print(bias.shape)
gpt = GPT2LMHeadModel.from_pretrained('distilgpt2')
if torch.cuda.is_available():
    gpt = gpt.cuda()
with torch.no_grad():
    gpt.transformer.h[-1].mlp.c_fc.bias += bias[0]

In [ ]:
with torch.no_grad():
    tokens = gpt(gpt_inputs['input_ids'])[0][0].argmax(-1)
print(gpt_tokenizer.decode(tokens))
print((tokens[:-1] != gpt_inputs['input_ids'][0, 1:]).sum().item())

In [ ]:
test_ids = gpt_tokenizer(prompt, return_tensors="pt").input_ids
if torch.cuda.is_available():
   test_ids = test_ids.to('cuda')

generated_ids = gpt.generate(
    test_ids,
    max_length = 128,
    num_return_sequences = 1,
    no_repeat_ngram_size = 5,
    num_beams = 5,
    eos_token_id = 50256,
    bad_words_ids = [[198]],
)

# 解码生成的文本
generated_text = gpt_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(generated_text[0])
print(len(generated_ids[0]))

# wiki

In [7]:
wiki = load_dataset('wikipedia', '20220301.simple',
          trust_remote_code = True)["train"].select(range(1000))

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

wikipedia.py:   0%|          | 0.00/36.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/134M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/205328 [00:00<?, ? examples/s]

In [9]:
def mapping(x):
  gpt_text = ['\n' + text.replace('\n',' ') for text in x['text']]
  gpt = gpt_tokenizer(gpt_text, return_tensors="pt", padding = 'max_length',
                      max_length = 128, truncation = True)
  return{
    'gpt_ids': gpt['input_ids'],
    'gpt_mask': gpt['attention_mask'],
  }

dataset = wiki.map(mapping, batched = True, load_from_cache_file = True,
                   remove_columns=['id','url','title','text'])
dataset.set_format('torch')
print(dataset)
train_size = len(dataset) - len(dataset) // 20
trainData, evalData = random_split(dataset, [train_size, len(dataset) // 20])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['gpt_ids', 'gpt_mask'],
    num_rows: 1000
})


In [17]:
import TextVAE
importlib.reload(TextVAE)
from TextVAE import TextVAE

tvae = TextVAE()
if torch.cuda.is_available():
    tvae = tvae.cuda()
    TextVAE.gpt.cuda()

In [ ]:
gpt_tokenizer.batch_decode(tvae(**trainData[0:3])[2].argmax(-1))

In [18]:
training_args = TrainingArguments(
    output_dir = "./results",
    save_strategy = "no",
    num_train_epochs = 100,
    weight_decay = 0,
    learning_rate = 5e-4,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    warmup_steps = 10,
    logging_steps = 32,
    # eval_strategy= "steps",
    # eval_steps = 10,
    fp16 = True,
    lr_scheduler_type = 'linear',
    max_grad_norm  = 1,
)

trainer = Trainer(
  model = tvae,
  args = training_args,
  train_dataset = trainData,
  # eval_dataset = evalData,
)
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


4.8824381828308105 820.1836547851562


Step,Training Loss
32,3.659400
64,3.357100
96,3.351200
128,3.339200
160,3.336200
192,3.323900
224,3.329500
256,3.310000
288,3.331100
320,3.321300


4.782739162445068 834.0145874023438
4.55057954788208 875.03955078125
4.2271623611450195 1004.9696044921875
3.884448766708374 1198.48095703125
3.782369613647461 1526.52734375
3.8703269958496094 2024.6328125
3.7445790767669678 2793.1953125
3.7228763103485107 3636.861572265625
3.626995086669922 4750.59765625
3.5358476638793945 6041.51416015625
3.693150520324707 7382.56640625
3.556807041168213 8893.146484375
3.5156424045562744 10393.380859375
3.4640309810638428 11658.943359375
3.481407642364502 13600.75
3.409691095352173 14793.7685546875
3.5842854976654053 15618.728515625
3.4539690017700195 17165.07421875
3.4934191703796387 17982.93359375
3.4562740325927734 19375.41015625
3.431575059890747 19844.5
3.406358242034912 20797.615234375
3.3892579078674316 22151.181640625
3.3544023036956787 23416.736328125
3.3582563400268555 23461.37109375
3.4518940448760986 23966.97265625
3.407942771911621 24344.23828125
3.53773832321167 25591.5390625
3.4277830123901367 26438.931640625
3.3728320598602295 26719.5

KeyboardInterrupt: 

In [ ]:
gpt = GPT2LMHeadModel.from_pretrained('distilgpt2').to('cuda')
input_ids = torch.tensor([[1,2,3]])
mask = torch.tensor([[1,1,0]])
gpt(input_ids, attention_mask = mask)[0] * mask.unsqueeze(-1)

In [ ]:
!python TextVAE.py

2024-11-16 06:29:34.124057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-16 06:29:34.143725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-16 06:29:34.149694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 06:29:35.414994: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
0.0


# TF_TextVAE

In [ ]:
!python TF_trainer.py --size 1 --batch_size 1 --warmup 10 --epoch 200 --ratio 1 --validation_freq 10000

In [ ]:
!python TF_TextVAE.py